## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [11]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [12]:
path_directorio='csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=1,day=8)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [16]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [19]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210118.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [20]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,209520,6800,216320,151404,0.699908,533,2021-01-17
1,Aragón,35415,1200,36615,25133,0.686413,0,2021-01-17
2,Asturias,35420,1200,36620,33572,0.916767,202,2021-01-18
3,Baleares,16740,600,17340,13750,0.792964,155,2021-01-17
4,Canarias,41310,1300,42610,42113,0.988336,327,2021-01-18
5,Cantabria,17700,500,18200,17724,0.973846,0,2021-01-16
6,Castilla y Leon,76740,2500,79240,67079,0.846530,124,2021-01-17
7,Castilla La Mancha,53100,1700,54800,38321,0.699288,0,2021-01-17
8,Cataluña,180020,5800,185820,145073,0.780718,167,2021-01-17
9,C. Valenciana,91450,2900,94350,93695,0.993058,152,2021-01-18


..hacemos una copia para hacerle las transformaciones necesarias:

In [27]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [28]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [29]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [30]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [31]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [32]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                     'Dosis entregadas Moderna','Dosis entregadas',
                     'Dosis administradas','% sobre entregadas',
                     'Total pauta completada',
                     'Fecha de la ultima vacuna registrada',
                     'source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [33]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [34]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [35]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,18/1/21,Andalucía,209520,6800,216320,151404,0.699908,533,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,18/1/21,Aragón,35415,1200,36615,25133,0.686413,0,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,18/1/21,Asturias,35420,1200,36620,33572,0.916767,202,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,18/1/21,Baleares,16740,600,17340,13750,0.792964,155,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,18/1/21,Canarias,41310,1300,42610,42113,0.988336,327,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,18/1/21,Cantabria,17700,500,18200,17724,0.973846,0,16/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,18/1/21,Castilla y Leon,76740,2500,79240,67079,0.846530,124,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,18/1/21,Castilla La Mancha,53100,1700,54800,38321,0.699288,0,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,18/1/21,Cataluña,180020,5800,185820,145073,0.780718,167,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,18/1/21,C. Valenciana,91450,2900,94350,93695,0.993058,152,18/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [36]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [37]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [38]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...
175,15/1/21,Navarra,17700.0,500.0,18200,9305,0.511264,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
176,15/1/21,País Vasco,47205.0,1400.0,48605,22464,0.462175,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
177,15/1/21,Ceuta,1005.0,200.0,1205,941,0.780913,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
178,15/1/21,Melilla,1005.0,200.0,1205,1230,1.020747,NaN,14/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [39]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [40]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas', 'Dosis administradas',
       '% sobre entregadas', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source',
       'Total pauta completada'],
      dtype='object')

In [41]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [43]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...
195,18/1/21,Navarra,17700.0,500.0,18200,11834,0.650220,0.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
196,18/1/21,País Vasco,47205.0,1400.0,48605,25526,0.525172,0.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
197,18/1/21,Ceuta,1005.0,200.0,1205,981,0.814108,29.0,NaN,17/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
198,18/1/21,Melilla,1005.0,200.0,1205,1289,1.069710,0.0,NaN,15/1/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [44]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [45]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)